In [6]:
from sklearn.utils import shuffle
import keras_tuner as kt
from tensorflow.compat.v1.metrics import true_positives, false_negatives
from keras.regularizers import l2
from keras.optimizers import Adam
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras.utils import image_dataset_from_directory

In [2]:
def load_model(file="model18.h5", hist="18_history.csv"):
    
    # load model and history
    
    import tensorflow as tf
    import pandas as pd
    import numpy as np
    model = tf.keras.models.load_model(file)
    history = pd.read_csv(hist)[['loss','binary_accuracy', 'val_loss', 'val_binary_accuracy']]
    return model, history

def get_data(train_path='images/train/', test_path='images/test/', valid_path='images/val/', size=(256, 256), batch_size=1):
    
    # preprocess train, test and valid sets
    
    from keras.utils import image_dataset_from_directory
    train = image_dataset_from_directory(directory=train_path,
                             label_mode='binary',
                             color_mode="rgb",
                             image_size=size,
                             batch_size=batch_size,
                             shuffle=True,
                             seed=77)
    test = image_dataset_from_directory(directory=test_path,
                             color_mode="rgb",
                             label_mode='binary',
                             image_size=size,
                             batch_size=batch_size,
                             shuffle=False,
                             seed=77)
    valid = image_dataset_from_directory(directory=valid_path,
                             color_mode="rgb",
                             label_mode='binary',
                             image_size=size,
                             batch_size=batch_size,
                             shuffle=False,
                             seed=77)
    return train, test, valid

def evaluate_model(model, n, valid, test):
    
    pd.DataFrame(model.history.history).plot(figsize=(15,5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.title(f'Model {n} Learning Curves')
    plt.show()
    print()
    print()
    print()
    print('Test set score')
    print()
    model.evaluate(test)
    print()
    print()
    print()
    print('Validation set score')
    print()
    model.evaluate(valid)

def dim_info(dimension):
    print("Dimension type: ", type(dimension))
    print("Dimension length: ", len(dimension))
    
def garson(A, B):
    # reference: https://csiu.github.io/blog/update/2017/03/29/day33.html
    """
    Computes Garson's algorithm
    A = matrix of weights of input-hidden layer (rows=input & cols=hidden)
    B = vector of weights of hidden-output layer
    """
    B = np.diag(B)

    # connection weight through the different hidden node
    cw = np.dot(A, B)
    cw = cw[1]
    # weight through node (axis=0 is column; sum per input feature)
    cw_h = abs(cw).sum(axis=0)

    # relative contribution of input neuron to outgoing signal of each hidden neuron
    # sum to find relative contribution of input neuron
    rc = np.divide(abs(cw), abs(cw_h))
    rc = rc.sum(axis=1)

    # normalize to 100% for relative importance
    ri = rc / rc.sum()
    return(ri)

def images_to_arrays(folder_path):
    X = []
    y = []
    for class_folder in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_folder)
        if not os.path.isdir(class_path):
            continue
        class_label = class_folder  # Assign the class folder name as the label
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (256, 256))  # Resize the image
                img_array = np.array(img, dtype=np.float32)  # Convert array values to float
                X.append(img_array)
                y.append(class_label)
    return np.array(X), np.array(y)

In [37]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Rescaling(1./255, input_shape=(256, 256, 3))) # input
        model.add(layers.MaxPooling2D(pool_size=2)) # maxpooling 1
        model.add(layers.Conv2D(64, kernel_size=2, kernel_regularizer=l2(0.01), activation="relu")) # hidden 1
        model.add(layers.MaxPooling2D(pool_size=2)) # maxpooling 2
        model.add(layers.Conv2D(64, kernel_size=2, kernel_regularizer=l2(0.01), activation="relu")) # hidden 2
        model.add(layers.MaxPooling2D(pool_size=2)) # maxpooling 3
        model.add(layers.Flatten())
        model.add(layers.Dropout(hp.Choice('rate',[0.5,0.6])))
        model.add(layers.Dense(1, activation="sigmoid")) # output
        model.compile(optimizer=Adam(learning_rate=0.0017),
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy'])
        return model

In [13]:
X_train, y_train = images_to_arrays("images/train")
X_valid, y_valid = images_to_arrays("images/val")

y_train[y_train == 'healthy'] = 0
y_train[y_train == 'sick'] = 1
y_train = y_train.astype('float32')

y_valid[y_valid == 'healthy'] = 0
y_valid[y_valid == 'sick'] = 1
y_valid = y_valid.astype('float32')

In [ ]:
#X_test, y_test = images_to_arrays("images/train")

# y_test[y_test == 'healthy'] = 0
# y_test[y_test == 'sick'] = 1
# y_test = y_test.astype('float32')

In [10]:
train = image_dataset_from_directory(directory='images/train/',
                             label_mode='binary',
                             color_mode="rgb",
                             image_size=(256, 256),
                             shuffle=True)
valid = image_dataset_from_directory(directory='images/val/',
                             color_mode="rgb",
                             label_mode='binary',
                             image_size=(256, 256),
                             shuffle=False,
                             seed=77)

Found 16930 files belonging to 2 classes.
Found 2115 files belonging to 2 classes.


In [33]:
tuner_gs = kt.GridSearch(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10)

INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [13]:
# 1st gs
tuner_gs.search(train, epochs=3, shuffle=True, validation_data=valid)

Trial 10 Complete [00h 08m 29s]
val_loss: 0.41179102659225464

Best val_loss So Far: 0.4015653729438782
Total elapsed time: 00h 57m 29s
INFO:tensorflow:Oracle triggered exit


In [24]:
gs_model_1 = tuner_gs.get_best_models()[0]

In [22]:
tuner_gs.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 06 summary
Hyperparameters:
pool_size: 2
filters: 64
kernel_size: 2
rate: 0.5
learning_rate: 0.0017
Score: 0.4015653729438782

Trial 08 summary
Hyperparameters:
pool_size: 2
filters: 64
kernel_size: 3
rate: 0.5
learning_rate: 0.0017
Score: 0.40299567580223083

Trial 04 summary
Hyperparameters:
pool_size: 2
filters: 32
kernel_size: 4
rate: 0.5
learning_rate: 0.0017
Score: 0.4042208194732666

Trial 00 summary
Hyperparameters:
pool_size: 2
filters: 32
kernel_size: 2
rate: 0.5
learning_rate: 0.0017
Score: 0.40968018770217896

Trial 09 summary
Hyperparameters:
pool_size: 2
filters: 64
kernel_size: 3
rate: 0.6
learning_rate: 0.0017
Score: 0.41179102659225464

Trial 07 summary
Hyperparameters:
pool_size: 2
filters: 64
kernel_size: 2
rate: 0.6
learning_rate: 0.0017
Score: 0.41249334812164307

Trial 01 summary
Hyperparameters:
pool_size: 2
filters: 32
kernel_size: 2
rate: 0.6


In [ ]:
# Hyperparameters:
# pool_size: 2
# filters: 64
# kernel_size: 2
# rate: 0.5
# learning_rate: 0.0017
# Score: 0.4015653729438782

In [ ]:
# 2nd gs
tuner_gs2 = kt.GridSearch(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10)
tuner_gs2.search(train, epochs=3, shuffle=True, validation_data=valid)

In [38]:
# 3nd gs
tuner_gs3 = kt.GridSearch(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10)
tuner_gs3.search(train, epochs=3, shuffle=True, validation_data=valid)

Trial 2 Complete [00h 07m 08s]
val_loss: 0.41039571166038513

Best val_loss So Far: 0.39599522948265076
Total elapsed time: 00h 14m 11s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Hyperparameters:
# rate: 0.5

In [39]:
tuner_gs3.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 00 summary
Hyperparameters:
rate: 0.5
Score: 0.39599522948265076

Trial 01 summary
Hyperparameters:
rate: 0.6
Score: 0.41039571166038513
